In [36]:
import pandas as pd
import re

In [37]:
filename=r"C:\Users\shrin\OneDrive\Documents\GitHub\Project-Expenditure\expense.xlsx"
sheet_name='2022'
xl= pd.read_excel(filename,sheet_name, header=1,usecols=range(4))

In [38]:
xl.columns

Index(['Date', 'Expense', 'Price', 'Total'], dtype='object')

In [39]:
xl.shape

(159, 4)

In [40]:
xl = xl.dropna(how="all")


In [41]:
xl.shape

(157, 4)

In [42]:
# 1. Define month regex for exact match
MONTH_PATTERN = re.compile(
    r"^(January|February|March|April|May|June|July|August|September|October|November|December)$",
    re.IGNORECASE
)

# 2. Detect rows in column A (index 0) that are month headers
is_month_header = xl.iloc[:, 0].astype(str).str.strip().str.match(MONTH_PATTERN)

# 3. Create a 'Month' column, forward-filling the last seen month header
xl["Month"] = xl.iloc[:, 0].where(is_month_header).ffill()

# 4. (Optional) Keep this for debugging
xl["IsMonthHeader"] = is_month_header

In [43]:
# Convert column C to numeric (coerce errors to NaN)
xl["ValueC"] = pd.to_numeric(xl.iloc[:, 2], errors="coerce")

# Optional: extract year from sheet name if it contains a 4-digit year
year_match = re.search(r"\b(20\d{2})\b", sheet_name)
xl["Year"] = int(year_match.group(1)) if year_match else None

In [44]:
# Clean Expense to normalize whitespace
xl["Expense_clean"] = xl["Expense"].astype(str).str.replace("\u00A0", " ").str.strip()

# Pattern: whole word month names or 'total' (case-insensitive)
exclude_pattern = re.compile(r"\b(total|(January|February|March|April|May|June|July|August|September|October|November|December))\b", re.IGNORECASE)

# Mask of rows to drop
mask_exclude = xl["Expense_clean"].str.contains(exclude_pattern, na=False)

# Debug what will be excluded
print("Excluding these rows:")
print(xl.loc[mask_exclude, ["Expense", "Expense_clean"]])

# Keep the rest
xl_cleaned = xl.loc[~mask_exclude].copy()

Excluding these rows:
      Expense Expense_clean
69      Total         Total
158  December      December


C:\Users\shrin\AppData\Local\Temp\ipykernel_9716\1278119971.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_exclude = xl["Expense_clean"].str.contains(exclude_pattern, na=False)


In [ ]:
# Group by month and sum column C
monthly_sum = (
    xl_cleaned
    .groupby("Month", dropna=True)["ValueC"]
    .sum()
    .reset_index(name="Sum_C")
)

month_order = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]
monthly_sum["Month"] = monthly_sum["Month"].str.capitalize()
monthly_sum["Month"] = pd.Categorical(monthly_sum["Month"], categories=month_order, ordered=True)
monthly_sum = monthly_sum.sort_values("Month").reset_index(drop=True)

# View result
print(monthly_sum)

      Month    Sum_C
0  November  29495.0
1       NaN  19917.5


In [46]:
# Save to CSV for manual inspection before grouping
output_debug_csv = f"debug_{sheet_name}_annotated.csv"
xl.to_csv(output_debug_csv, index=False)
print(f"Intermediate annotated data written to: {output_debug_csv}")

Intermediate annotated data written to: debug_2022_annotated.csv
